In [2]:
import pandas as pd
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import clear_output
import plotly.graph_objects as go

In [3]:
TICKERS_CSV = "data/averages/average_hot_scores.csv"  
OUTPUT_DIR = "output/cash_flow"  

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
df_tickers = pd.read_csv(TICKERS_CSV)

tickers = (
    df_tickers["symbol"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)

In [ ]:
columns_to_plot = [
    "Free Cash Flow",
    "Financing Cash Flow",
    "Investing Cash Flow",
    "Operating Gains Losses",
    "Operating Cash Flow",
    "Gain Loss On Investment Securities"
]

for ticker in tickers:
    try:
        t = yf.Ticker(ticker)
        cf = t.quarterly_cashflow.T  # transpose for easier plotting

        if cf.empty:
            print(f"{ticker}: No cash flow data")
            continue

        # Convert index to datetime
        cf.index = pd.to_datetime(cf.index)

        # Create Plotly figure
        fig = go.Figure()

        # Add line per cash flow component
        for col in columns_to_plot:
            if col in cf.columns:
                fig.add_trace(go.Scatter(
                    x=cf.index,
                    y=cf[col],
                    mode="lines+markers",
                    name=col,
                    line=dict(width=2),
                    marker=dict(size=6),
                    hovertemplate='<b>%{x|%Y-%m-%d}</b><br>%{y:$,.0f}<extra></extra>'
                ))

        # Layout
        fig.update_layout(
            title=f"{ticker} — Quarterly Cash Flow Components",
            xaxis_title="Quarter End Date",
            yaxis_title="Cash Flow (USD)",
            template="plotly_dark",
            hovermode="x unified",
            margin=dict(l=80, r=40, t=80, b=60),
            height=500,
            font=dict(size=13),
            legend=dict(
                x=-0.15,
                y=1,
                xanchor="left",
                yanchor="top"
            )
        )

        chart_path = os.path.join(OUTPUT_DIR, f"{ticker}.html")
        fig.write_html(chart_path, include_plotlyjs="cdn")
        print(f"{ticker}: chart saved → {chart_path}")
    except Exception as e:
        print(f"{ticker}: error → {e}")

RYTM: chart saved → output/cash_flow\RYTM.html
TFX: chart saved → output/cash_flow\TFX.html
ARES: chart saved → output/cash_flow\ARES.html
SAIC: chart saved → output/cash_flow\SAIC.html
ULTA: chart saved → output/cash_flow\ULTA.html
KEN: chart saved → output/cash_flow\KEN.html
ASH: chart saved → output/cash_flow\ASH.html
DSGX: chart saved → output/cash_flow\DSGX.html
URBN: chart saved → output/cash_flow\URBN.html
JBTM: chart saved → output/cash_flow\JBTM.html
SNPS: chart saved → output/cash_flow\SNPS.html
UNF: chart saved → output/cash_flow\UNF.html
BMRN: chart saved → output/cash_flow\BMRN.html
MDGL: chart saved → output/cash_flow\MDGL.html
TPL: chart saved → output/cash_flow\TPL.html
MDB: chart saved → output/cash_flow\MDB.html
MTN: chart saved → output/cash_flow\MTN.html
ELPC: chart saved → output/cash_flow\ELPC.html
GE: chart saved → output/cash_flow\GE.html
KYMR: chart saved → output/cash_flow\KYMR.html
TWO-PB: chart saved → output/cash_flow\TWO-PB.html
FTAI: chart saved → output/